In [8]:
import os
import numpy as np
import pandas as pd
import glob
import re

import m2stitch


In [2]:
cluster = ""
workdir = "/workdir/bmg224/harvard_dental/manuscript/code"
os.chdir(cluster + workdir)
os.getcwd()

'/local/workdir/bmg224/harvard_dental/manuscript/code'

In [3]:
out_dir = "../outputs/segmentation_2024_03_07"
out_fmt_register = out_dir + "/{date}/{sn}/registration"
reg_fmt = out_fmt_register + '/{sn}_registered_{m}.npy'
tile_info_fmt = out_fmt_register + '/{sn}_tile_info.csv'


In [4]:
sn = '2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01'
date = '2022_12_16'

In [15]:
tile_info_fn = tile_info_fmt.format(date=date, sn=sn)
reg_glob = reg_fmt.format(date=date, sn=sn, m='*')
reg_fns = glob.glob(reg_glob)
Ms = [int(re.search('(?<=_registered_)\d+',fn)[0]) for fn in reg_fns]
reg_fns = [x for _, x in sorted(zip(Ms, reg_fns))]
reg_fns

['../outputs/segmentation_2024_03_07/2022_12_16/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01/registration/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01_registered_0.npy',
 '../outputs/segmentation_2024_03_07/2022_12_16/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01/registration/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01_registered_1.npy',
 '../outputs/segmentation_2024_03_07/2022_12_16/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01/registration/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01_registered_2.npy',
 '../outputs/segmentation_2024_03_07/2022_12_16/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01/registration/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_fov_01_registered_3.npy',
 '../outputs/segmentation_2024_03_07/2022_12_16/2022_12_16_harvardwelch_patient_14_tooth_14_aspect_MB_depth_sub_

In [16]:
stack_sums = []
for fn in reg_fns:
    stack = np.load(fn)
    stack_sums.append(np.sum(stack, axis=2))
stack_sums = np.dstack(stack_sums).transpose(2,0,1)
stack_sums.shape

In [22]:
tile_info = pd.read_csv(tile_info_fn)
nrows = tile_info['rows'].values[0]
ncols = tile_info['cols'].values[0]
M = nrows * ncols
print(nrows, ncols)


4 4


In [27]:
rows, cols

(array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]),
 array([0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3]))

In [43]:
stack_test = stack_sums[[0,1,3,4],:,:]

In [44]:
rows = [0,0,1,1]
cols = [0,1,0,1]
result_df, _ = m2stitch.stitch_images(
    stack_test, 
    rows, 
    cols, 
    pou=10,
    ncc_threshold=0.01,
    row_col_transpose=False,
)
print(result_df["y_pos"])

print(result_df["x_pos"])



100%|██████████| 4/4 [00:12<00:00,  3.14s/it]


AssertionError: 

In [35]:
cols = np.tile(np.arange(ncols), nrows)
rows = np.repeat(np.arange(nrows), ncols)
result_df, _ = m2stitch.stitch_images(
    stack_sums, 
    rows, 
    cols, 
    pou=10,
    ncc_threshold=0.1,
    row_col_transpose=False,
)

print(result_df["y_pos"])
# the absolute y (second last dim.) positions of the tiles
print(result_df["x_pos"])
# the absolute x (last dim.) positions of the tiles

# stitching example
result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()

size_y = stack_sums.shape[1]
size_x = stack_sums.shape[2]

stitched_image_size = (
    result_df["y_pos2"].max() + size_y,
    result_df["x_pos2"].max() + size_x,
)
stitched_image = np.zeros_like(stack_sums, shape=stitched_image_size)
for i, row in result_df.iterrows():
    stitched_image[
        row["y_pos2"] : row["y_pos2"] + size_y,
        row["x_pos2"] : row["x_pos2"] + size_x,
    ] = stack_sums[i]

 12%|█▎        | 2/16 [00:08<00:57,  4.13s/it]


KeyboardInterrupt: 

In [25]:
stitched_image.shape

(13452, 2195)